# Corporate FinTech Assignment 2

#### Name and e-mail

Mads Duelund Dorka, mador17@student.sdu.dk

Max Festersen Hansen, maxfh20@student.sdu.dk

Mathias Eriksen, merik17@student.sdu.dk

Daniel Lindberg, dlind16@student.sdu.dk

Emilie Bruun Therp, emthe15@student.sdu.dk

#### Imports

In [7]:
import pprint # To print "stuff" pretty
import pandas as pd # To work with dataframes and math functions
import numpy as np # to work with math
import statsmodels.api as sm # To use OLS
import json # To work with json files
from textblob import TextBlob # To do Naïve Bayes classifification
from textblob.classifiers import NaiveBayesClassifier
from datetime import datetime # To format strings as dates
import textstat # To get fog gunning values
#from matplotlib.pylab import plt # To plot plots
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import classification_report, confusion_matrix
import statsmodels.api as sm

#### Enviroment and Data imports

In [8]:
filename = 'icoData_19092018.json'
with open(filename) as json_data:
    icoData = json.load(json_data) #Load data into a list

#### Formatting data 

In [9]:
icoData = [x for x in icoData if not len(x) == 1]  # Filter list - remove empty dirs

In [10]:
icoDataFiltPersonNum = [] # For person instance
icoDataFiltReview = [] # For reviews
icoDataFiltReviewNum = [] # For a unique identifier for reviews
icoDataFiltTeamRating = [] # The team rating;
icoDataFiltVisionRating = [] # The vision rating;
icoDataFiltProductRating = [] # The product rating;
icoDataFiltOverallRating = [] # Overall rating;
icoDataFiltAmountRaised = [] # Amount raised;
icoDataFiltSuccess = [] # Successes

i = 0
personNum = 0
reviewNum = 0

#### Loop all of icoData

In [11]:
for person in icoData:
    personNum += 1 # Count next person
    # Filter end-dates after 19-10-2018
    endDate = person['dates']["icoEnd"].split(" ")[0]
    if endDate != '0000-00-00' and datetime.strptime(endDate, "%Y-%m-%d") < datetime.strptime("2018-10-19", "%Y-%m-%d"):
        validDate = True # If the date is a valid date, and before 2018-10-19
    else:
        validDate = False
    if validDate:
        for rating in person['ratings']:
            if "review" in rating and len(rating["review"]) > 0:
                reviewNum += 1 # Count valid review
                # Save ratings review instance
                icoDataFiltPersonNum.append(personNum) # Person number for data validation
                icoDataFiltReviewNum.append(reviewNum) # Review number as uniqe classification
                icoDataFiltReview.append(rating['review']) # Review used in Exercise 2a-d and Exercise 3 part 1-3
                icoDataFiltTeamRating.append(rating["team"]) # Team rating from review
                icoDataFiltVisionRating.append(rating["vision"]) # Vision rating from review
                icoDataFiltProductRating.append(rating["product"]) # Product rating from review
                icoDataFiltOverallRating.append(person["rating"]) # Overall rating from person (NOTE: NOT REVIEW LEVEL)
                amountRaised = person["finance"]["raised"] # Amount raised
                icoDataFiltAmountRaised.append(amountRaised) # Append amount raised
                icoDataFiltSuccess.append(1 if amountRaised > 0 else 0) # Success (= dummy (1) if amount raised larger 0).
    i+=1 # Increment instance counter

#### Format as dataframe

In [12]:
reviewDt = pd.DataFrame({   '#review': icoDataFiltReviewNum,
                            'Person ID': icoDataFiltPersonNum,
                            'Review':icoDataFiltReview,
                            'Team rating':icoDataFiltTeamRating,
                            'Vision rating':icoDataFiltVisionRating,
                            'Product rating':icoDataFiltProductRating,
                            'Overall Rating':icoDataFiltOverallRating,
                            'Amount Raised':icoDataFiltAmountRaised,
                            'Success':icoDataFiltSuccess
                         })

## Exercise 1

Natural language processing is a concept where the automated generation and understanding of natural human languages are studied. The concept of readability is how a reader can understand a written text, moreover how hard or easy a text is to read. The readability measures depend on various “variables” i.e. Complexity of the words, the syntax, or the length of the sentences, depending on which model you are using. In the end, you get a value, which tells the reader how hard the text is to read.

One way to measure, the readability measure is by the Fog index. When using the Fog index, the test aims to show the reader the difficulty level of the text. The level/index refers to the number of years of education a person needs to have to fully understand the text on the first reading.
 
 
To calculate the Fog index, the following components must be known.
-    Average sentence length, percentage of long words and the sum of average sentence length and the percentage of long words.
 
However, when checking for long words, there are some exceptions i.e.Company names, combines short-words and short three-syllable words.

When writing in business language, you should aim for a score around 20-25, and in an office report a score of 30-35. NLP is used more than ever in financial research. Financial firms are using the NLP to parse textual data. Instead of having employees to check the textual data, the NLP are faster and more accurate than humans are. When using the NLP, companies are able to filter and analyze data faster. This allows companies to follow their stocks very closely and sell them if they discover that the company is budgeting with a loss in their earnings report.
 
NLP can improve decision-making and speed inside financial organizations with three instances. Automation, Data enrichment, and Search and discovery.

## Exercise 2

### a)

Median and 50% quartile is effectivly the same. We asume that the exercise needs 75% quartile instead of a redundant median. This assumtion is applicable for all of the assignment.


Mean:

We would expect the mean to be around zero, as we would expect approximately the same amount of positive and negative words. However, given the type of data, and that people often are positive when it comes to new projects, one could argue that the mean probably is leaning to the positive side.


Minimum and maximum:

Initially we would expect the minimum value to be -1 and the maximum value to 1, however as seen in class, it is quite unusual to see a negative word valued -1, hence one might expect it to be a little larger.

25%:

We would expect the 25th quantile to be negative, as we would argue that there would be a minimum of 25% negative comments.


50% / Median:

As argued above, we would expect the median to be slightly above 0, as we would expect people to have a positive and open mind when it comes to new projects.


75%:

We would expect the 75th quantile to be around 0.3-0.5, as we would expect more positively weighted words, however we would argue that people rarely use words which are extremely positive, and hence weighted as 1. People are more likely to use less weighted positive words, even in extremely positive cases.

#### Math calculater  

The Describe funktionen from pandas is used to return a overview containing count, mean, standard deviation, minimim value, 25%, median, 75% og maximum value

#### Calculate the polarity score 

In [13]:
sentimetsPolarity = []
sentimetsSubjectivity = []
for review in icoDataFiltReview: # For every review in the filtered reviews
    tb = TextBlob(review) # Peform sentiment analasys
    sentimetsPolarity.append(tb.sentiment[0]) # Get polarity score
    sentimetsSubjectivity.append(tb.sentiment[1]) # Get subjectivity score

#### Add to dataframe

In [14]:
reviewDt["Polarity"] = sentimetsPolarity
reviewDt["Subjectivity"] = sentimetsSubjectivity

#### Analyzes

In [15]:
reviewDtSemanticInfo = reviewDt[['Polarity']].describe()
#median = reviewDt[['Polarity']].median()
#reviewDtSemanticInfo = reviewDtSemanticInfo.transpose()

In [16]:
pprint.pprint(reviewDtSemanticInfo)

          Polarity
count  5483.000000
mean      0.300281
std       0.267197
min      -0.700000
25%       0.118803
50%       0.265714
75%       0.466667
max       1.000000


#### Interpretation /discussion of economic aspects of the result 

Mean:

We calculated a mean of 0.3, which shows that people tend to be more positive in the data. This is in compliance with what we discussed in the expectations, namely the fact that people tend to lean to the positive side when it comes to new projects.


Minimum and maximum:

We see a minimum of -0.7 and a maximum of 1. This is not in compliance with the initial thought, however it does fit what we have seen in class, with very few negative words actually being weighted as -1-


25%:

We see that the 25th quantile for the data is 0.1188, this is not what we expected. This deems to show that people tend to write fewer negative words than what we would expect. However, this is in compliance with what we mentioned above, with people tending to be more positive about startups.


50% / Median:

Same logic as with the 25th quantile. We expected the median to be slightly lower than the 0.2657 that we found. However, this might be reasoned by people being more positive about startups.


75%:

The 75th quantile is in compliance with what we initially thought. It is in the high end of what was expected, but nonetheless in compliance. This does make quite good sense, as people aren’t that likely to use very heavily weighted positive words, but instead opts for less heavily weighted words.

### b)

#### Expectation section

We expect that the Fog index will be in the high end of the scale. This is because it is mostly university employees, professors, and people trading with crypto currencies that reads and comments on these projects.

#### Math calculator 

Fog Index = 0.4 *(average # of words per sentence + 100 * percent of complex words)

#### Get Fog gunning values 

In [20]:
fogValues = []
for review in icoDataFiltReview: # For every review in the filtered reviews
    fogValues.append(textstat.gunning_fog(review)) # Get and append fog index for review

#### Add to dataframe 

In [21]:
reviewDt["Fog Index"] = fogValues

#### Analyzis 

In [22]:
reviewDtFogInfo = reviewDt[['Fog Index']].describe()

In [23]:
pprint.pprint(reviewDtFogInfo)

         Fog Index
count  5483.000000
mean     11.983903
std       9.750480
min       0.000000
25%       7.315000
50%      10.340000
75%      14.640000
max     156.910000


#### Interpretation /discussion of economic aspects of the result 

The relevant results shows the opposite of what we expected. When calculating the Fog index, we see that the mean is around 12. From the results, we also see that the 50% quantile is lower than the mean. From this, we can see that reviews about the median are a lot higher than the median, which also visible when looking at max.


However, when looking at the results it does make sense that the Fog index is low. This could be explained by the fact that the given text is short reviews. Reviews tend to be short and simple, making it easy to read.

### c)

####  Expectation section

We expect that the overall rating will have a positive correlation between amount raised and the success of the project. However, we do not expect that the other variables correlate. The rating variables will most likely be more correlated than the other variables.

#### Math calculater 

A correlation matrix measure of the linear association between two variables. It has a value between -1 and 1 where:

•    -1 indicates a perfectly negative linear correlation between two variables

•    0 indicates no linear correlation between two variables

•    1 indicates a perfectly positive linear correlation between two variables


The further away the correlation coefficient is from zero, the stronger the relationship between the two variables.

corr function from pandas is used on the datasetet to create a correlation matrix.

#### Execution of task 

In [25]:
corrMatrix = reviewDt[['Polarity',
                       'Fog Index',
                       'Team rating',
                       'Vision rating',
                       'Product rating',
                       'Overall Rating',
                       'Amount Raised',
                       'Success']].corr()
                         # Missing from data: the team rating; the vision rating; the product rating; overall rating; amount raised; success (= dummy (1) if amount raised larger 0).
pprint.pprint(corrMatrix)

                Polarity  Fog Index  Team rating  Vision rating  \
Polarity        1.000000  -0.224758     0.420646       0.442749   
Fog Index      -0.224758   1.000000    -0.096129      -0.097844   
Team rating     0.420646  -0.096129     1.000000       0.756970   
Vision rating   0.442749  -0.097844     0.756970       1.000000   
Product rating  0.444003  -0.132969     0.764593       0.822642   
Overall Rating  0.312247  -0.013937     0.676591       0.631365   
Amount Raised   0.009342   0.083689     0.036685       0.039661   
Success         0.115465  -0.039221     0.184371       0.166338   

                Product rating  Overall Rating  Amount Raised   Success  
Polarity              0.444003        0.312247       0.009342  0.115465  
Fog Index            -0.132969       -0.013937       0.083689 -0.039221  
Team rating           0.764593        0.676591       0.036685  0.184371  
Vision rating         0.822642        0.631365       0.039661  0.166338  
Product rating        1.00

#### Interpretation /discussion of economic aspects of the result 

The correlation between Fog index, and everything but the amount raised is negative. This does intuitively make sense, as a high Fog index would make for more complicated reading, and hence you could assume that if the Fog index is high, less people would be able to actually use the text, and hence it would be rated lesser.

However, people with a higher degree, might be able to invest more money, and hence it might make sense that there is a positive correlation here. Polarity generally has a positive correlation to the other variable, except fog index.


Overall rating has a positive correlation to success. Then the overall rating increases, then there is a greater chance that the project will be successful.

### d)

#### i) 

#### Expectation section

Ordinary least square regression estimates the relationship between one or more independent variables and a dependent variable by minimizing the sum of the squares in the difference between the observed and the predicted values.

#### Run an OLS regression of the amount raised on polarity score. Fog index, the team rating, the vision rating, the product rating. Overall rating. Interpret your results.

#### OLS Regression 

In [26]:
def ols_amountRaisedOn(X): # Give predictor value - does ols with amount raised as predictor - returns fitted ols estimate
    X = sm.add_constant(X) # add constant to predictor
    y = reviewDt[["Amount Raised"]] # response value
    est = sm.OLS(y, X) # Create model from response, y and predictor X
    est = est.fit() # fitting model
    #X_prime = np.linspace(X.iloc[:, 1].min(), X.iloc[:, 1].max(), 100)[:, np.newaxis]
    #X_prime = sm.add_constant(X_prime) 
    #y_hat = est.predict(X_prime)
    #fig = plt.xlabel("Amount Raised") 
    #fig = plt.ylabel(X.columns[1]) 
    #fig = plt.plot(X_prime[:, 1], y_hat, 'r', alpha=0.9)
    #plt.show()
    return est.summary() # return params

#### OLS:  Amount raised on polarity score, fog index, the team rating, the vision rating, the product rating, overall rating. 

In [27]:
olsResult = ols_amountRaisedOn(reviewDt[["Polarity", "Fog Index","Team rating", "Vision rating", "Product rating", "Overall Rating"]])
pprint.pprint(olsResult)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:          Amount Raised   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     8.854
Date:                Sun, 21 Mar 2021   Prob (F-statistic):           1.24e-09
Time:                        23:35:34   Log-Likelihood:            -1.0768e+05
No. Observations:                5483   AIC:                         2.154e+05
Df Residuals:                    5476   BIC:                         2.154e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

#### Interpretation /discussion of economic aspects of the result. 

From the OLS regression we see that the Fog Index has a P-value of 0.000, hence we reject the impact of the Fog value on the amount raised. We see that the other variables are statistically significant on a 95% significance level for the amount raised. 

This does intuitively make sense, as Fog index only explains the difficulty of the text, and not wether it is positively or negatively weighted.

#### ii)

#### Logistic regression
#### Expectation section

Looking at the logit regression, we can reject everything but the be vision rating and the product rating on a 95% significance level.

In [28]:
x = reviewDt[["Polarity", "Fog Index","Team rating", "Vision rating", "Product rating", "Overall Rating"]]
y = reviewDt["Success"]
model = sm.Logit(y,x)
result=model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.640781
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                Success   No. Observations:                 5483
Model:                          Logit   Df Residuals:                     5477
Method:                           MLE   Df Model:                            5
Date:                Sun, 21 Mar 2021   Pseudo R-squ.:                 0.02351
Time:                        23:35:36   Log-Likelihood:                -3513.4
converged:                       True   LL-Null:                       -3598.0
Covariance Type:            nonrobust   LLR p-value:                 1.079e-34
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Polarity           0.3473      0.125      2.783      0.005       0.103       0.592
Fog Index        

#### Interpretation /discussion of economic aspects of the result.  

Hence, we see that vision rating and product rating seems to be the only relevant factors for success, however all but Fog index are relevant for the amount raised.

## Exercise 3

#### Expectation section 

We expect the results from randomly selected data from our own classifiers result to be similar to the result from Textblob, but we do expect our trained data to be more precice.

### part 1 

#### Selecting 150 random reviews 

In [29]:
reviewDtSample = reviewDt.sample(n=150, random_state=666)

#### reviewDtSample["n"] =   range(150) 

In [30]:
reviewDtSample["class"]=["negative", "negative", "neutral", "neutral", "positive", "positive", "negative", "neutral", "negative", "positive", "positive", "neutral", "positive", "neutral", "negative", "neutral", "positive", "positive", "positive", "positive", "positive", "positive", "positive", "negative", "positive", "neutral", "neutral", "negative", "positive", "positive", "positive", "neutral", "neutral", "positive", "positive", "positive", "neutral", "positive", "positive", "positive", "positive", "positive", "positive", "positive", "neutral", "positive", "negative", "neutral", "positive", "negative", "negative", "positive", "positive", "positive", "neutral", "positive", "positive", "positive", "negative", "negative", "positive", "neutral", "negative", "positive", "negative", "positive", "positive", "positive", "negative", "positive", "negative", "positive", "positive", "neutral", "positive", "positive", "positive", "neutral", "neutral", "positive", "neutral", "negative", "positive", "negative", "neutral", "negative", "negative", "negative", "positive", "negative", "positive", "positive", "positive", "negative", "neutral", "positive", "positive", "positive", "positive", "neutral", "positive", "positive", "positive", "positive", "positive", "negative", "neutral", "negative", "positive", "positive", "positive", "positive", "negative", "positive", "positive", "positive", "negative", "positive", "negative", "negative", "negative", "positive", "positive", "positive", "positive", "positive", "negative", "positive", "positive", "positive", "positive", "positive", "positive", "negative", "negative", "negative", "positive", "positive", "positive", "negative", "positive", "neutral", "positive", "positive", "positive", "negative", "positive", "neutral", "positive", "positive" ]

In [31]:
reviewDtSampleTraining = reviewDtSample.iloc[0:99].copy(deep=False)
reviewDtSampleTest = reviewDtSample.iloc[100:149].copy(deep=False)

### Part 3 

#### Train classifier using textblob 

In [32]:
cl = NaiveBayesClassifier(reviewDtSampleTraining[["Review", "class"]].values.tolist()) # Train on training set

#### Test on testdata 

In [33]:
reviewDtSampleTests = []

for review in reviewDtSampleTest["Review"].values:
    reviewDtSampleTests.append(cl.classify(review))

In [34]:
reviewDtSampleTest.__setitem__('classCl', reviewDtSampleTests)

In [37]:
reviewDtClaccuracy = np.where(reviewDtSampleTest["classCl"] == reviewDtSampleTest["class"], 1, 0)

In [38]:
reviewDtClaccuracyPercent = sum(reviewDtClaccuracy)/len(reviewDtClaccuracy)

#### Interpretation /discussion of economic aspects of the result.  

The accuracy of the trained model is:  65.31 %.

The accuracy of the training model is relatively low. This could be due the complexity of the reviews posibly has caused human error and a machine with confusing instructions.

### Part 3 

#### Calculate the polarity score 

In [39]:
sentimetsPolarityCl = []
sentimetsSubjectivityCl = []
for review in icoDataFiltReview: # For every review in the filtered reviews
    tb = TextBlob(review, classifier=cl) # Peform sentiment analasys
    sentimetsPolarityCl.append(tb.sentiment[0]) # Get polarity score
    sentimetsSubjectivityCl.append(tb.sentiment[1]) # Get subjectivity score

#### Add to dataframe 

In [40]:
reviewDt["PolarityCL"] = sentimetsPolarityCl
reviewDt["SubjectivityCL"] = sentimetsSubjectivityCl

#### Analyzis 

In [41]:
reviewDtCLSemanticInfo = reviewDt[['PolarityCL']].describe()

In [42]:
pprint.pprint(reviewDtCLSemanticInfo)

        PolarityCL
count  5483.000000
mean      0.300281
std       0.267197
min      -0.700000
25%       0.118803
50%       0.265714
75%       0.466667
max       1.000000


#### Interpretation /discussion of economic aspects of the result.  

The polarity scores with with our classifier are exactly the same as Textblob's classifier.